In [ ]:
gtsrb_path = kagglehub.dataset_download('meowmeowmeowmeowmeow/gtsrb-german-traffic-sign')
print(gtsrb_path)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import accuracy_score
from tensorflow import keras
from keras import layers
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time
import datetime
from tqdm import tqdm

from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n-cls.pt')

In [ ]:
model.train(
    data='/content/gtsrb-german-traffic-sign/', 
    imgsz=32,
    batch=128,
    epochs=66,
    mixup=0.2,
    cutmix=0.2,
    device=0
)

In [ ]:
model = YOLO('runs/classify/train22/weights/best.pt')

########This Code Section Not Mine, Took this Code Block from Kaggle for Using Testing Dataset Properly########
y_test = pd.read_csv('../kaggle/input/gtsrb-german-traffic-sign/Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

with tf.device('/GPU:0'):
    for img in imgs:
        image = Image.open('../kaggle/input/gtsrb-german-traffic-sign/'+img)
        image = image.resize([32, 32])
        data.append(np.array(image))

X_test=np.array(data)

with tf.device('/GPU:0'):
    t0 = time.time()
    preds = np.argmax(model.predict(X_test, batch_size=128, verbose=1), axis=-1)
    t1 = time.time()

acc = accuracy_score(labels, preds)
print(f"Accuracy: {acc:.4f}")

total_time = t1 - t0
num_images = X_test.shape[0]
avg_time_per_image = (total_time / num_images) * 1000

print(f"Total inference time for {num_images} images: {total_time:.2f} seconds")
print(f"Average inference time per image: {avg_time_per_image:.4f} ms")

########################################################################################